In [24]:
import spacy 
import pickle 
import random
import warnings

In [25]:
train_data = pickle.load(open('train_data.pkl', 'rb'))
len(train_data)


200

In [35]:
nlp = spacy.blank('en')
def train_model(train_data): 
    if "ner" not in nlp.pipe_names: 
        ner = nlp.create_pipe("ner") 
        nlp.add_pipe(ner, last = True) 
  
    for _, annotation in train_data: 
        
        for ent in annotation['entities']: 
            ner.add_label(ent[2])
            nlp = spacy.blank('en') 
    other_pipes = [pipe for pipe in nlp.pipe_names if  pipe != 'ner'] 
    
    with nlp.disable_pipes(*other_pipes): # only  train NER 
        optimizer = nlp.begin_training() 
            
        for itn in range(10): 
            print("Statring iteration " + str(itn)) 
            random.shuffle(train_data) 
            losses = {} 
            index = 0 
        for text, annotations in train_data: 
            try: 
                 nlp.update( 
                 [text], # batch of texts 
                 [annotations], # batch of  annotations 
                 drop=0.2, # dropout - make it  harder to memorise data 
                 sgd=optimizer, # callable to  update weights 
                 losses=losses) 
            except Exception as e: 
                     pass 
        print(losses)

In [36]:
train_model(train_data)

UnboundLocalError: local variable 'nlp' referenced before assignment

In [ ]:
nlp.to_disk('nlp_model')

nlp_model = spacy.load('nlp_model') 

train_data[0][0] 


#result :

doc = nlp_model(train_data[0][0]) 
for ent in doc.ents: 
    print(f'{ent.label_.upper():{30}}- {ent.text}')